# Import Libraries

In [1]:
import configparser
import pandas as pd

# Read Config File

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')
input_directory = config['DEFAULT']['Input-Files-Directory']
Import_Option_File = config['DEFAULT']['Import-Option-File']
extracted_census_files = config['DEFAULT']['Extracted-Census']
population_file = config['DEFAULT']['Population-data']
name_file = config['DEFAULT']['Name']
output_file = config['DEFAULT']['Output-File']

# Read Data

In [3]:
extracted_collection_dictionary = {}
for file in extracted_census_files.split(";"):
    extracted_collection_dictionary[file] = pd.read_csv('{}/{}.csv'.format(input_directory, file), sep=',', encoding='utf-8')
pop_df = pd.read_csv('{}/{}.csv'.format(input_directory, population_file), sep=',', encoding='utf-8')

# Scale the counts based on the relevant population

In [4]:
def scale_columns(extracted_collection_dictionary, import_option_df, pop_df):
    updated_extracted_collection_dictionary = {} # store scaled extracted values
    for index, row in import_option_df.iterrows():
        for table in extracted_collection_dictionary.keys():
            if row['Table Number'] in table:
                agg = row['Aggregate']
                if agg == 'none': # if Aggregate is none, then no scaling. 
                    updated_extracted_collection_dictionary[table] = extracted_collection_dictionary[table]   
                    continue
                else:
                    scale_value = row['Aggregate']
                # merge variable data with population data, we use merging for matching index
                extracted_collection_dictionary[table]['SA1_7DIGITCODE_2016'] = extracted_collection_dictionary[table]['SA1_7DIGITCODE_2016'].astype(int)
                merged_df = pd.merge(extracted_collection_dictionary[table], pop_df[['SA1_7DIGITCODE_2016', scale_value]], on='SA1_7DIGITCODE_2016')
                # scale count by related population
                for c in merged_df.loc[:, merged_df.columns != 'SA1_7DIGITCODE_2016'].columns:
                    merged_df[c] = merged_df[c]/merged_df[scale_value]
                # remove the related column and update dictionary with scaled DF
                merged_df.drop(columns=[scale_value], inplace=True)
                updated_extracted_collection_dictionary[table] = merged_df        
    return updated_extracted_collection_dictionary

In [5]:
import_option_df = pd.read_csv('{}.txt'.format(Import_Option_File), delimiter = ",", comment='#')
scaled_extracted_collection_dictionary = scale_columns(extracted_collection_dictionary, import_option_df, pop_df)

# Combine Dataframes

In [6]:
def combine_dataframe(updated_extracted_collection_dictionary):
    """ Merge all dataframe on SA1 code """
    combined_df = pd.DataFrame(columns=['SA1_7DIGITCODE_2016'])
    n = 0
    for df in updated_extracted_collection_dictionary:
        if n == 0:
            combined_df = updated_extracted_collection_dictionary[df]
        else:
            combined_df = pd.merge(combined_df, 
                                   updated_extracted_collection_dictionary[df],
                                  on = 'SA1_7DIGITCODE_2016')
        n += 1
    return combined_df

In [7]:
def remove_null_rows(df):
    """ remove rows where contains a null value """
    df = df.dropna(how='any')
    return df

In [8]:
combined_df = combine_dataframe(scaled_extracted_collection_dictionary)
combined_df = remove_null_rows(combined_df)

# Rename Columns

In [9]:
def rename_columns(name_df, df):
    """ renmae columns """
    for index, row in name_df.iterrows():
        if row['oldname'] in df.columns:
            df.rename(columns={row['oldname']:row['newname']}, inplace=True)

    return combined_df  

In [10]:
name_df = pd.read_csv('{}.txt'.format(name_file), delimiter = ",", comment='#')
renamed_df = rename_columns(name_df, combined_df)
# merge with population DF
final_df = pd.merge(renamed_df, pop_df, left_on= 'code', right_on='SA1_7DIGITCODE_2016', how='left')
# sort by sa1 code
final_df.sort_values(by=['code'], inplace=True)
# drop columns
final_df.drop(columns=['P_ID_NS_Tot' ,'P_Tot_Tot_x', 'P_Tot_Occu_ID_NS', 'P_Tot_Tot_y', 'Not Stated', 'Ancestry Total', 'SA1_7DIGITCODE_2016'], inplace=True)

# Save Result

In [11]:
final_df.to_csv('{}.csv'.format(output_file), sep=',', encoding='utf-8', index=False)